In [3]:
# !pip install wxPython

In [1]:
import wx
import math 

import import_ipynb
import G360_Function as func


### SUELO Y MATERIALES

In [2]:
#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# INPUTS FIJOS
input_ = {
    'Cohesion' : None ,
    'Sigma_adm' : None ,
    'Gamma_suelo' : None ,
    'Phi' : None
}
input_str = {        
    'Cohesion':    f'C     Cohesión [kN/m\u00b2]:' ,
    'Sigma_adm':   f'σ_cap Capacidad admisible [KPa]:' ,
    'Gamma_suelo': f'γ_s   Densidad del suelo [kN/m\u00b3]:' ,
    'Phi' :        f'φ     Ángulo de fricción interna [°]:' 
}

inputOP_ = {
    'Betha' : None ,
    'Gamma_suelo_rell' : None ,
    'fy' : None ,
    'fprimac' : None ,
    'Suelo_sumergido' : None,
    'Prof_desprecio' : None,
    'Comportamiento_suelo': None 
}

inputOP_str = {
    'Betha' :               f'β        Ángulo de arrancamiento [°]:' ,
    'Gamma_suelo_rell' :    f'γ_rell   Densidad del suelo de relleno [kN/m\u00b3]:' ,
    'fy' :                  f'fy       Fluencia del acero [MPa]:' ,
    'fprimac' :             f'f\'c     Capacidad a compresión del concreto [MPa]:' ,
    'Suelo_sumergido' :     f'         ¿La cimencación está sumergida?' ,
    'Prof_desprecio' :      f'         Profundidad de desprecio (PANAMA) [m]:',
    'Comportamiento_suelo': f'         Comportamiento GRANULAR / COHESIVO:'
}
inputOP_default = [
    lambda: round(2/3*input_['Phi'],2) , # Betha
    lambda: input_['Gamma_suelo'] , # Gamma_suelo_rell
    lambda: 420 , # fy
    lambda: 21 , # fprimac
    lambda: 'SUMERGIDO' , # SECO == 0 , SUMERGIDO == 1
    lambda: 0 , #Prof_desprecio 0.6 m
    lambda: 'COHESIVO' # GRANULAR == 0 , COHESIVO == 1
]

# Variables que tienen selección:____
vrbl_Seleccion = ['Suelo_sumergido' , # SECO  SUMERGIDO
                  'Comportamiento_suelo' # GRANULAR  COHESIVO
                 ]
Valores_ComboBox = {'SECO' : 0, 'SUMERGIDO' : 1,
                    'GRANULAR': 0, 'COHESIVO' : 1}

# CALCULOS AGREGADOS:
# kp
# Gamma_concr

# ACOMODAR CADENAS DE TEXTO:
_max_value = func.len_values(input_str | inputOP_str)
_margen = 5
for k,v in input_str.items():
    input_str[k] += ' '*(_max_value - len(v) + _margen)  
for k,v in inputOP_str.items():
    inputOP_str[k] += ' '*(_max_value - len(v) + _margen)  

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
#variables del suelo y materiales
class MyFrameInputsSuelo(wx.Frame):
    def __init__(self, parent, title):
        super(MyFrameInputsSuelo, self).__init__(parent, title = title, size = (600, 700))  # tamaño de la ventana
        # font1 = wx.Font(12, wx.FONTFAMILY_MODERN, wx.FONTSTYLE_NORMAL, wx.FONTWEIGHT_NORMAL, faceName="Consolas")
        # self.SetFont(font1)
####
# PANEL
        self.panel = wx.Panel(self)
        self.vbox = wx.BoxSizer(wx.VERTICAL)
        self.grid_sizer = wx.FlexGridSizer(len(input_), 2, 10, 10)  # :, 2 columnas

        # INPUTS FIJOS
        # CREAR ETIQUETAS Y CAJAS entry DE input_ EN EL self.grid_sizer
        self._dicc_input_ = {}
        for k,v in input_.items():
            _txt = self._dicc_input_[f'txt_{str(k)}']   = wx.StaticText(self.panel, label = f'{input_str[k]}')
            _box = self._dicc_input_[f'entry_{str(k)}'] = wx.TextCtrl(self.panel)
            # AÑADIRLOS
            self.grid_sizer.Add(_txt, 0, wx.ALIGN_CENTER_VERTICAL)
            self.grid_sizer.Add(_box, 0, wx.EXPAND)
####
# HIDDEN PANEL
        self.hidden_panel = wx.Panel(self.panel)
        self.vbox_hidden = wx.BoxSizer(wx.VERTICAL)
        self.grid_hidden = wx.FlexGridSizer(len(inputOP_), 2, 10, 10)  # :, 2 columnas
        # self.hidden_panel.Hide() 
        self.hidden_panel.Show() # MOSTRAR PARA CONFIGURAR EL SIZE FINAL

        self.vbox_hidden.Add(self.grid_hidden, 0, flag = wx.ALL | wx.EXPAND, border = 10)
        self.hidden_panel.SetSizer(self.vbox_hidden)

        # INPUTS OPCIONALES
        # CREAR ETIQUETAS Y CAJAS entry DE ENTRADA EN EL self.grid_hidden
        self._dicc_inputOP_ = {}
        
        _entradas_sobreescribir = vrbl_Seleccion
        for k,v in inputOP_.items():
            if k in _entradas_sobreescribir:
                opciones = list(Valores_ComboBox.keys())
                _txt = self._dicc_inputOP_[f'txt_{str(k)}']   = wx.StaticText(self.hidden_panel, label = f'{inputOP_str[k]}')
                _box = self._dicc_inputOP_[f'entry_{str(k)}'] = wx.ComboBox(self.hidden_panel, choices=opciones)
            else:
                _txt = self._dicc_inputOP_[f'txt_{str(k)}']   = wx.StaticText(self.hidden_panel, label = f'{inputOP_str[k]}')
                _box = self._dicc_inputOP_[f'entry_{str(k)}'] = wx.TextCtrl(self.hidden_panel)
            # AÑADIRLOS
            self.grid_hidden.Add(_txt, 0, wx.ALIGN_CENTER_VERTICAL)
            self.grid_hidden.Add(_box, 0, wx.EXPAND)

####
# BOTONES
        self.btn_toggle = wx.Button(self.panel, label="▲ Menos Variables Opcionales")
        self.btn_toggle.Bind(wx.EVT_BUTTON, self.on_toggle)
        
        self.btn_save = wx.Button(self.panel, label="   OK   ")
        self.btn_save.Bind(wx.EVT_BUTTON, self.on_save)
####
#LAYOUT
        # ORGANIZACIÓN DE ELEMENTOS EN EL LAYOUT
        self.vbox.Add(self.grid_sizer,   0, flag = wx.ALL | wx.CENTER,       border=10) #wx.EXPAND
        self.vbox.Add(self.btn_toggle,   0, flag = wx.ALIGN_CENTER | wx.TOP, border=10)
        self.vbox.Add(self.hidden_panel, 0, flag = wx.ALL | wx.CENTER,       border=10)
        self.vbox.Add(self.btn_save,     0, flag = wx.ALIGN_CENTER | wx.TOP, border=10)

        self.panel.SetSizer(self.vbox)
####
# METODOS Y FUNCIONES
    def on_save(self, event):
        # CARGAR VALORES
        for k,v in input_.items():
            input_[k]   = float( self._dicc_input_[f'entry_{str(k)}'].GetValue()
                             )
        for k,v in inputOP_.items():
            _get_value = self._dicc_inputOP_[f'entry_{str(k)}'].GetValue()
            try:
                inputOP_[k] = float( _get_value )
            except ValueError:
                inputOP_[k] = Valores_ComboBox[_get_value]
####            
        # AGREGAR VARIABLES ADICIONALES Y ACTUALIZAR UNIDADES DE ANGULOS
        inputOP_['kp'] = round(math.tan(math.radians(45 + input_['Phi']/2))**2, 2)
        input_['Phi']   = math.radians(input_['Phi'])
        
        inputOP_['Betha'] = math.radians(inputOP_['Betha'])
        inputOP_['Gamma_concr'] = 24 - inputOP_['Suelo_sumergido'] * 10
        
        self.Close()
        
    def on_toggle(self, event):
        for k,v in input_.items():
            input_[k]   = self._dicc_input_[f'entry_{str(k)}'].GetValue()
        for k,v in inputOP_.items():
            inputOP_[k] = self._dicc_inputOP_[f'entry_{str(k)}'].GetValue()

        # SI LAS ENTRADAS TIENEN CONTENIDO, THEN:        
        if not "" in list(input_.values()):
            # CARGAR INPUTS DIJOS
            for k,v in input_.items():
                input_[k]   = float(v)
            # CARGAR VALORES POR DEFECTO DE LAS VARIABLES OPCIONALES
            _inputOP_keys = list(inputOP_.keys())
            for i in range(len(inputOP_)):
                inputOP_[ _inputOP_keys[i-1] ] = func.dict_default(inputOP_default[i-1])
            # INGRESAR VALORES POR DEFECTO EN LAS CAJAS DE ENTRADA entry
            for k,v in inputOP_.items():
                self._dicc_inputOP_[f'entry_{str(k)}'].SetValue(str(v))

            # MOSTRAR U OCULTAR self.hidden_panel
            if self.hidden_panel.IsShown():
                self.hidden_panel.Hide()
                self.btn_toggle.SetLabel("▼ Más Variables Opcionales")
            else:
                self.hidden_panel.Show()
                self.btn_toggle.SetLabel("▲ Menos Variables Opcionales")
        else:
            self.btn_toggle.SetLabel("▼ *INGRESAR INPUTS* ▼")
        # Reajustar la interfaz
        self.Layout()

    def init_toggle(self, event):
        self.hidden_panel.Hide()
        self.btn_toggle.SetLabel("▼ Más Variables Opcionales")    

####
# CLASE QUE EJECUTA LA VENTANA MyFrameInputsSuelo()
class VentanaInputsSuelo(wx.App):
    def OnInit(self):
        self.frame = MyFrameInputsSuelo(None, "Variables input")
        self.frame.Show(True)
        # EJECUTAR LA FUNCIÓN init_toggle PARA OCULTAR EL hidden_panel
        wx.CallAfter(self.frame.init_toggle, None)  # Llamamos a on_toggle() para ocultar el panel
        return True
####
# EJECUTAR
if __name__ == "__main__":
    app = VentanaInputsSuelo(False)
    app.MainLoop()

In [3]:
import pprint
pprint.pprint(input_)
pprint.pprint(inputOP_)

{'Cohesion': None, 'Gamma_suelo': None, 'Phi': None, 'Sigma_adm': None}
{'Betha': None,
 'Comportamiento_suelo': None,
 'Gamma_suelo_rell': None,
 'Prof_desprecio': None,
 'Suelo_sumergido': None,
 'fprimac': None,
 'fy': None}


### REACCIONES MATRIZ

In [ ]:
# Reacciones matriz
import wx
import wx.grid

f_diag_ort_casos = None
# JcGG_FLAG: MODIFICAR EL ORDEN PARA QUE SALGA IGUAL AL EXCEL

class MyFrameReaccionesDiagOrt(wx.Frame):
    def __init__(self, parent, title):
        super(MyFrameReaccionesDiagOrt, self).__init__(parent, title=title, size=(650, 250))
        
        self.panel_reacciones = wx.Panel(self)
        self.grid_matrix = wx.FlexGridSizer(2, 2, 0, 0)

        #CREAR ETIQUETAS
        self.lab_diag = wx.StaticText(self.panel_reacciones, label = f'Sistema Diagonal [kN]')
        self.lab_ort = wx.StaticText(self.panel_reacciones, label = f'Sistema Ortogonal [kN]')
        
        #GRIDS
        self.grid_diag = wx.grid.Grid(self.panel_reacciones)
        self.grid_diag.CreateGrid(4, 3) 

        self.grid_ort = wx.grid.Grid(self.panel_reacciones)
        self.grid_ort.CreateGrid(4, 3)

        #TAMAÑO DE LAS CELDAS
        for col in range(self.grid_diag.GetNumberCols()):
            self.grid_diag.SetColSize(col, 80)  # Ajusta el tamaño de las columnas a 100 píxeles
        for row in range(self.grid_diag.GetNumberRows()):
            self.grid_diag.SetRowSize(row, 25)  # Ajusta el tamaño de las filas a 30 píxeles

        for col in range(self.grid_ort.GetNumberCols()):
            self.grid_ort.SetColSize(col, 80)  # Ajusta el tamaño de las columnas a 100 píxeles
        for row in range(self.grid_ort.GetNumberRows()):
            self.grid_ort.SetRowSize(row, 25)  # Ajusta el tamaño de las filas a 30 píxeles  
            
        #BLOQUEAR EL AJUSTE DE TAMÑOS
        self.grid_diag.EnableDragColSize(False)  # Desactiva la capacidad de redimensionar las columnas
        self.grid_diag.EnableDragRowSize(False)

        #UBICACIÓN EN grid_matrix
        self.grid_matrix.Add(self.lab_diag, wx.ALIGN_CENTER_VERTICAL)
        self.grid_matrix.Add(self.lab_ort, wx.ALIGN_CENTER_VERTICAL)
        
        self.grid_matrix.Add(self.grid_diag, wx.ALIGN_CENTER_VERTICAL)
        self.grid_matrix.Add(self.grid_ort, wx.ALIGN_CENTER_VERTICAL)

        
        # ETIQUETAS grid_matrix
        # self.grid_diag.SetColLabelValue(0, "CASO DE CARGA")
        self.grid_diag.SetColLabelValue(0, "FV")
        self.grid_diag.SetColLabelValue(1, "FL")
        self.grid_diag.SetColLabelValue(2, "FT")
        #
        self.grid_diag.SetRowLabelValue(0, "Compresión")
        self.grid_diag.SetRowLabelValue(1, "Tracción")
        self.grid_diag.SetRowLabelValue(2, "Max Longitudinal")
        self.grid_diag.SetRowLabelValue(3, "Max Transversal")

        self.grid_ort.SetColLabelValue(0, "FV")
        self.grid_ort.SetColLabelValue(1, "FL")
        self.grid_ort.SetColLabelValue(2, "FT")   
        #
        self.grid_ort.HideRowLabels()

        #BOTON DE GUARDADO
        self.btn_reacciones = wx.Button(self.panel_reacciones, label = '   OK   ')
        self.btn_reacciones.Bind(wx.EVT_BUTTON, self.save_reacciones)
        
        # CONFIGURAR EL LAYOUT vbox
        self.vbox = wx.BoxSizer(wx.VERTICAL)
        self.vbox.Add(self.grid_matrix, 1, flag = wx.EXPAND | wx.ALL, border = 5)
        self.vbox.Add(self.btn_reacciones, 0, flag = wx.ALIGN_CENTER | wx.TOP)

        
        self.panel_reacciones.SetSizerAndFit(self.vbox)
        self.Show()


    def save_reacciones(self, event):
        lista_reacciones_diag = [self.grid_diag.GetCellValue(f, c) for f in range(self.grid_diag.GetNumberRows()) for c in range(self.grid_diag.GetNumberCols())]
        lista_reacciones_ort = [self.grid_ort.GetCellValue(f, c) for f in range(self.grid_ort.GetNumberRows()) for c in range(self.grid_ort.GetNumberCols())]

        valores_reacciones_diag = [(0 if i == "" else float(i)) for i in lista_reacciones_diag]
        valores_reacciones_ort = [(0 if i == "" else float(i)) for i in lista_reacciones_ort]

        fv_diag_comp, fl_diag_comp, ft_diag_comp, fv_diag_tr, fl_diag_tr, ft_diag_tr, fv_diag_long, fl_diag_long, ft_diag_long, fv_diag_trnsv, fl_diag_trnsv, ft_diag_trnsv = valores_reacciones_diag
        fv_ort_comp, fl_ort_comp, ft_ort_comp, fv_ort_tr, fl_ort_tr, ft_ort_tr, fv_ort_long, fl_ort_long, ft_ort_long, fv_ort_trnsv, fl_ort_trnsv, ft_ort_trnsv = valores_reacciones_ort
        
        self.Close()

        
# Clase principal de la aplicación que hereda de wx.App
class VentanaDeReacciones(wx.App):
    def OnInit(self):
        self.frame = MyFrameReaccionesDiagOrt(None, "Variables input")
        self.frame.Show(True)
        return True

# Ejecutar la aplicación
if __name__ == "__main__":
    app = VentanaDeReacciones(False)
    app.MainLoop()

### GEOMETRÍA DE LA TORRE

In [ ]:
# Geometría torre
input_g = {
    'ancho_stub' : None ,
    'long_stub' : None ,
    'sep_patas' : None ,
    'ang_bidireccional' : None ,
    'ang_tridimensional' : None ,
}
input_g_str = {
    'ancho_stub' : f'Ancho del stub incluyendo ancla [m]:' ,
    'long_stub' : f'Longitud del stub [m]:' ,
    'sep_patas' : f'Separación de las patas entre PR [m]:' ,
    'ang_bidireccional' : f'Ángulo bidireccional de la pata [°]' ,
    'ang_tridimensional' : f'Ángulo tridimensional de la pata [°]' 
}

### GEOMETRÍA CIMENTACIÓN

In [1]:
# Geometría Cimentación


import wx
import math 

import import_ipynb
import G360_Function as func

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#

input_z = {
    'recub_ped' : 5/100 ,
    'recub_dis' : 5/100 ,
    'lista_ped' : '0.25, 0.50, 0.75, 1.00'
}
input_z = {
    'recub_ped': 0.05,
    'recub_dis': 0.05, 
    'lista_ped': '0.25, 0.50, 0.75, 1.00', 
    'pata_comp':  1,
    'pata_trac':  4,
    'Esp_solado': 0.05,
    'Geom_Ped' = 0   #  CUADRADO == 0 , CIRCULAR == 1
    }
input_z_str = {
    'recub_ped' : f'Recubrimiento pedestal [m]' ,
    'recub_dis' : f'Recubrimiento losa inferior [m]' ,
    'lista_ped' : f'Lista de altura de pedestales separados por "coma : ," [m]',
    'pata_comp' : f' txt',
    'pata_trac':  f' txt',
    'Esp_solado': f' txt',
    'Geom_Ped' : f'txt'
}

input_zOP = {
    'P' : None ,
    'A' : None  ,
    'E' : None ,
    'EP' : None 
}
input_zOP_str = {
    'P' :  f'Profundidad  [m]' ,
    'A' :  f'Ancho losa [m]',
    'E' :  f'Espesor losa [m]',
    'EP' : f'Espesor Pedestal [m]'
}

# ACOMODAR CADENAS DE TEXTO:
_max_value = func.len_values(input_z_str | input_zOP_str)
_margen = 5
for k,v in input_z_str.items():
    input_z_str[k] += ' '*(_max_value - len(v) + _margen)  
for k,v in input_zOP_str.items():
    input_zOP_str[k] += ' '*(_max_value - len(v) + _margen)  

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
#variables de la cimentación
class MyFrameInputsGeometriaZapata(wx.Frame):
    def __init__(self, parent, title):
        super(MyFrameInputsGeometriaZapata, self).__init__(parent, title = title, size = (700, 700))  # tamaño de la ventana
        # font1 = wx.Font(12, wx.FONTFAMILY_MODERN, wx.FONTSTYLE_NORMAL, wx.FONTWEIGHT_NORMAL, faceName="Consolas")
        # self.SetFont(font1)
####
# PANEL
        self.panel = wx.Panel(self)
        self.vbox = wx.BoxSizer(wx.VERTICAL)
        self.grid_sizer = wx.FlexGridSizer(len(input_z), 2, 10, 10)  # :, 2 columnas

        # INPUTS FIJOS
        # CREAR ETIQUETAS Y CAJAS entry DE input_z EN EL self.grid_sizer
        self._dicc_input_ = {}
        for k,v in input_z.items():
            _txt = self._dicc_input_[f'txt_{str(k)}']   = wx.StaticText(self.panel, label = f'{input_z_str[k]}')
            _box = self._dicc_input_[f'entry_{str(k)}'] = wx.TextCtrl(self.panel, value = str(input_z[k]))
            # AÑADIRLOS
            self.grid_sizer.Add(_txt, 0, wx.ALIGN_CENTER_VERTICAL)
            self.grid_sizer.Add(_box, 1, wx.EXPAND)
        
####
# HIDDEN PANEL
        self.hidden_panel = wx.Panel(self.panel)
        self.vbox_hidden = wx.BoxSizer(wx.VERTICAL)
        self.grid_hidden = wx.FlexGridSizer(len(input_zOP), 2, 10, 10)  # :, 2 columnas
        # self.hidden_panel.Hide() 
        self.hidden_panel.Show() # MOSTRAR PARA CONFIGURAR EL SIZE FINAL

        self.vbox_hidden.Add(self.grid_hidden, 0, flag = wx.ALL | wx.EXPAND, border = 10)
        self.hidden_panel.SetSizer(self.vbox_hidden)

        # INPUTS OPCIONALES
        # CREAR ETIQUETAS Y CAJAS entry DE ENTRADA EN EL self.grid_hidden
        self._dicc_inputOP_ = {}
        
        for k,v in input_zOP.items():
            _txt = self._dicc_inputOP_[f'txt_{str(k)}']   = wx.StaticText(self.hidden_panel, label = f'{input_zOP_str[k]}')
            _box = self._dicc_inputOP_[f'entry_{str(k)}'] = wx.TextCtrl(self.hidden_panel)
            # AÑADIRLOS
            self.grid_hidden.Add(_txt, 0, wx.ALIGN_CENTER_VERTICAL)
            self.grid_hidden.Add(_box, 0, wx.EXPAND)

####
# BOTONES
        self.btn_toggle = wx.Button(self.panel, label="▲ Geometría específica")
        self.btn_toggle.Bind(wx.EVT_BUTTON, self.on_toggle)
        
        self.btn_save = wx.Button(self.panel, label="   OK   ")
        self.btn_save.Bind(wx.EVT_BUTTON, self.on_save)
####
#LAYOUT
        # ORGANIZACIÓN DE ELEMENTOS EN EL LAYOUT
        self.vbox.Add(self.grid_sizer,   0, flag = wx.ALL | wx.CENTER,       border=10) #wx.EXPAND
        self.vbox.Add(self.btn_toggle,   0, flag = wx.ALIGN_CENTER | wx.TOP, border=10)
        self.vbox.Add(self.hidden_panel, 0, flag = wx.ALL | wx.CENTER,       border=10)
        self.vbox.Add(self.btn_save,     0, flag = wx.ALIGN_CENTER | wx.TOP, border=10)

        self.panel.SetSizer(self.vbox)
####
# METODOS Y FUNCIONES
    def on_save(self, event):
        # CARGAR VALORES
        for k,v in input_z.items():
            try:
                input_z[k]   = float( self._dicc_input_[f'entry_{str(k)}'].GetValue() )
            except ValueError:
                cadena = input_z['lista_ped']
                input_z['lista_ped'] = [float(x.strip()) for x in cadena.split(',')]
                           
        for k,v in input_zOP.items():
            try:
                input_zOP[k]   = float( self._dicc_inputOP_[f'entry_{str(k)}'].GetValue() )
            except Exception:
                input_zOP[k]   = str( self._dicc_inputOP_[f'entry_{str(k)}'].GetValue() )

        # ITERAR? , THEN:        
        input_zOP['Iterar'] = 1 if "" in list(input_zOP.values()) else 0
        self.Close()
        
    def on_toggle(self, event):
        for k,v in input_z.items():
            input_z[k]   = self._dicc_input_[f'entry_{str(k)}'].GetValue()
        for k,v in input_zOP.items():
            input_zOP[k] = self._dicc_inputOP_[f'entry_{str(k)}'].GetValue()

        # SI LAS ENTRADAS TIENEN CONTENIDO, THEN:        
        if not '' in list(input_z.values()):
            # CARGAR INPUTS FIJOS
            for k,v in input_z.items():
                try:
                    input_z[k]   = float(v)
                except ValueError:
                    input_z[k]   = str(v)          

            # MOSTRAR U OCULTAR self.hidden_panel
            if self.hidden_panel.IsShown():
                self.hidden_panel.Hide()
                self.btn_toggle.SetLabel("▼ Geometría específica")
            else:
                self.hidden_panel.Show()
                self.btn_toggle.SetLabel("▲ Ver menos")
        else:
            self.btn_toggle.SetLabel("▼ *INGRESAR INPUTS* ▼")
        # Reajustar la interfaz
        self.Layout()

    def init_toggle(self, event):
        self.hidden_panel.Hide()
        self.btn_toggle.SetLabel("▼ Geometría específica")    

####
# CLASE QUE EJECUTA LA VENTANA MyFrameInputsGeometriaZapata()
class VentanaInputsGeometriaZapata(wx.App):
    def OnInit(self):
        self.frame = MyFrameInputsGeometriaZapata(None, "Variables input")
        self.frame.Show(True)
        # EJECUTAR LA FUNCIÓN init_toggle PARA OCULTAR EL hidden_panel
        wx.CallAfter(self.frame.init_toggle, None)  # Llamamos a on_toggle() para ocultar el panel
        return True
####
# EJECUTAR
if __name__ == "__main__":
    app = VentanaInputsGeometriaZapata(False)
    app.MainLoop()

['', '', '', '', 1]


In [4]:
import pprint
pprint.pprint(input_z)
pprint.pprint(input_zOP)

{'lista_ped': [0.25, 0.5, 0.75, 1.0], 'recub_dis': 0.05, 'recub_ped': 0.05}
{'A': '', 'E': '', 'EP': '', 'Iterar': 1, 'P': ''}
